## Cross-Bucket Tests on all Acc/Dec and Random Fragments Kalman Filter

### Part A: (1) Import Fragment Data from MATLAB, (2) Save Them in a .pickle file, and (3) Load Them In

###### (HOWEVER, YOU DON'T NEED TO LOAD THEM IN AGAIN SINCE STEP 1 ALREADY DOES THAT FOR YOU!)

In [1]:
## (Configuration) Allows you to return multiple variables from a single cell ##
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Allows you to import files from another folder in current directory ## 
import os 
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle
import sys

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho
from Neural_Decoding.metrics import get_R2_parts

#Import decoder functions
from Neural_Decoding.decoders import KalmanFilterDecoder

### Part B: Preprocessing Decoder Data

In [3]:
## Import Data ##
# We'll load in position data and derive velocity and acceleration from it 

# Specify Fragment Types to be used for this anaylsis 
frag_type = ['AD', 'Rand'] 
# Specify folder where MATLAB data is stored
folder = '/Users/rbhatt6/Documents/MATLAB/' 

 #locals()["sortIn"+frag_type[i]] = io.loadmat(folder+'cleanedSortIn.mat')
for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    locals()[input] = io.loadmat(folder + input + '.mat')

    output = "sortOut_" + frag_type[i]
    locals()[output] = io.loadmat(folder + output + '.mat')

    locals()[input] = np.squeeze(list(locals()[input].values())[3])
    locals()[output] = np.squeeze(list(locals()[output].values())[3])

In [4]:
## Format Kinematics Data (outputs) ##
# For the Kalman filter, we use the position, velocity, and acceleration as outputs.
# Ultimately, we are only concerned with the goodness of fit of velocity, but using them all as covariates helps performance.

for i in range(len(frag_type)):
    # Pulling local variables into new, temp variables
    output = locals()["sortOut_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(output)): # Number of buckets (i.e. 16 or 8)
        nFrags = output[j][0].shape[0]
        temp2 = []
        for k in range(0, nFrags, 1): # Number of fragments #output[0][0].shape[0]
            vel_X = float(output[j][0][k])
            vel_Y = float(output[j][1][k])
            acc_X = float(output[j][2][k])
            acc_Y = float(output[j][3][k])
            pos_X = float(output[j][4][k])
            pos_Y = float(output[j][5][k])
            temp = [vel_X, vel_Y, acc_X, acc_Y, pos_X, pos_Y]
            temp2.append(np.array(temp))
            #locals()[decoder_output][j][k].append(np.array(temp))
        #locals()[decoder_output][j].append(np.array(temp2))
        locals()[decoder_output].append(np.array(temp2))
        #temp = np.array(np.concatenate((vel_X, vel_Y, acc_X, acc_Y, outputX[j], outputY[j]),axis=1))

### Part C: Partitioning and Running the Kalman Filter on the Same Buckets 

In [ ]:
# (NEW) CROSS-VALIDATED WITHIN-BUCKET TEST (for all 16 buckets)
# Doing a 10-fold cross validation procedure

# for loop through 10 folds then 16 buckets

for i in range(len(frag_type)):

   # Pulling local variables 
   input = locals()["sortIn_"+frag_type[i]]
   output = locals()["decoder_output_"+frag_type[i]]

   # Creating a local variable to hold final training and testing data 
   final_X_train = "final_X_train_" + frag_type[i]
   final_y_train = "final_y_train_" + frag_type[i]
   final_X_test = "final_X_test_" + frag_type[i]
   final_y_test = "final_y_test_" + frag_type[i]
   locals()[final_X_train] = []
   locals()[final_y_train] = []
   locals()[final_X_test] = []
   locals()[final_y_test] = []

   idx_list = [0,1,2,3,4,5,6,7,8,9]
   # final_X_train = []
   # final_y_train = []
   # final_X_test = []
   # final_y_test = []

   # Splitting data for a single bucket into 10 folds - (j) is the left-out fold for testing
   for j in range(0,10):
      test_idx = j
      train_idx = idx_list.copy()
      train_idx.pop(j)

      temp_X_train = []
      temp_y_train = []
      temp_X_test = []
      temp_y_test = []
      
      for k in range(len(input)):
         X_split = np.array_split(input[k], 10)
         y_split = np.array_split(output[k], 10)

         # Index the list of indices to create a list of np ndarrays for training (9 arrays) and testing (1 array)
         X_train = [X_split[i] for i in train_idx]
         X_test = X_split[j]
         y_train = [y_split[i] for i in train_idx]
         y_test = y_split[j]

         # Concatenate the list of 9 arrays for the training sets 
         X_train = np.concatenate(X_train, axis=0)
         y_train = np.concatenate(y_train, axis=0)
         
         temp_X_train.append(X_train)
         temp_y_train.append(y_train)
         temp_X_test.append(X_test)
         temp_y_test.append(y_test)

      locals()[final_X_train].append(temp_X_train)
      locals()[final_y_train].append(temp_y_train)
      locals()[final_X_test].append(temp_X_test)
      locals()[final_y_test].append(temp_y_test)

## Check for Decoder Accuracy
### Training on 15 buckets and Testing on 1 bucket instead

In [15]:
# Only using velocity 
## Format Kinematics Data (outputs) ##
# For the Kalman filter, we use the position, velocity, and acceleration as outputs.
# Ultimately, we are only concerned with the goodness of fit of velocity, but using them all as covariates helps performance.

for i in range(len(frag_type)):
    # Pulling local variables into new, temp variables
    output = locals()["sortOut_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(output)): # Number of buckets (i.e. 16 or 8)
        nFrags = output[j][0].shape[0]
        temp2 = []
        for k in range(0, nFrags, 1): # Number of fragments #output[0][0].shape[0]
            vel_X = float(output[j][0][k])
            vel_Y = float(output[j][1][k])
            temp = [vel_X, vel_Y]
            temp2.append(np.array(temp))
        locals()[decoder_output].append(np.array(temp2))



In [ ]:
listArrays = final_y_test_AD[0][0:3]
listArrays

sampleList = [1,2]

slipList = [listArrays[i] for i in sampleList]
slip = np.concatenate(slipList, axis=0)

len(slipList)
slip.shape

In [17]:
# Within-Bucket Test with 10-fold Cross-Validation 
from Neural_Decoding.runModelsKF import run_model_kf_test

for i in range(len(frag_type)):

    # Pulling local variables 
    frag_X_train =  locals()["final_X_train_" + frag_type[i]]
    frag_y_train = locals()["final_y_train_" + frag_type[i]]
    frag_X_test = locals()["final_X_test_" + frag_type[i]]
    frag_y_test = locals()["final_y_test_" + frag_type[i]]

    # Creating a local variable to hold predicted outputs for each frag type AND their trained models
    parts = "pred_parts_" + frag_type[i]
    models = "more_trained_models_" + frag_type[i]
    locals()[parts] = []
    locals()[models] = []

    for cross in range(0,10):
        X_train = frag_X_train[cross]
        y_train = frag_y_train[cross]
        X_test = frag_X_test[cross]
        y_test = frag_y_test[cross]
        
        curr_R2s, curr_models = run_model_kf_test(X_train, y_train, X_test,y_test, "parts")
        locals()[parts].append(curr_R2s)
        locals()[models].append(curr_models)

In [18]:
# Compute Within-Bucket combined XY_FVAF 
# For velocity only, although all kinematic variables should be used for model-fitting and predicted for.
from Neural_Decoding.metrics import compute_XY_FVAF

# 'pred_parts_AD' was thankfully also a list of arrays containing the R2s

for i in range(len(frag_type)):

    # Pulling local variables (computed R2s for parts)
    R2s_parts = locals()["pred_parts_" + frag_type[i]]

    # Creating a local variable to hold final XY_FVAFs
    XY_FVAFs = "XY_FVAFs_" + frag_type[i]
    locals()[XY_FVAFs] = []

    for j in range(0,10):
        curr_fold = R2s_parts[j]

        curr_fold_XY_FVAFs = []
        for k in range(0,16):
            #curr_bucket = Kalman_R2s_combined[i]
            vel_x_nom = curr_fold[k][0][0] # dim = (curr_bucket, nom, x_vel)
            vel_x_denom = curr_fold[k][1][0] # dim = (curr_bucket, denom, x_vel)
            vel_y_nom = curr_fold[k][0][1] # dim = (curr_bucket, nom, y_vel)
            vel_y_denom = curr_fold[k][1][1] # dim = (curr_bucket, denom, y_vel)

            XY_FVAF = compute_XY_FVAF(vel_x_nom,vel_x_denom,vel_y_nom,vel_y_denom)
            curr_fold_XY_FVAFs.append(XY_FVAF)

        locals()[XY_FVAFs].append(curr_fold_XY_FVAFs)

In [21]:
XY_FVAFs_Rand

[[0.18562900105147018,
  0.11542580820668069,
  0.14123323914564356,
  0.1313249399239147,
  0.1974403498660241,
  0.0029681292454749597,
  0.20849075893882985,
  0.05959470863125993,
  0.14046321475732793,
  0.17872962490255695,
  0.13008646104343358,
  0.1927348175191157,
  0.17399811911594398,
  0.13387619839980203,
  0.15954570513771016,
  0.14636177034759967],
 [0.18084049553881032,
  0.12176030103338886,
  0.11298028646593272,
  0.23069395927643754,
  0.12237708560429039,
  0.14633100004262722,
  0.1269378710978284,
  0.155954339747563,
  0.1814380294186857,
  0.1319703924676895,
  0.14234371114066724,
  0.17866867982857026,
  0.1364621895977428,
  0.1001449985368188,
  0.12425811864372249,
  0.19306484986961447],
 [0.08422297037867799,
  0.18584488075654115,
  0.09239303075379424,
  0.10232419107315383,
  0.14380886267649573,
  0.2101714375241409,
  0.16997241843795674,
  0.22036072467993728,
  0.1462147725184949,
  0.1334439546994769,
  0.1167580351026859,
  0.15990113833570563

## Training and Testing Models on Same Bucket

In [22]:
# (NEW) Within-Bucket Test with 10-fold Cross-Validation 
from Neural_Decoding.runModelsKF import run_model_kf_cv

for i in range(len(frag_type)):

    # Pulling local variables 
    frag_X_train =  locals()["final_X_train_" + frag_type[i]]
    frag_y_train = locals()["final_y_train_" + frag_type[i]]
    frag_X_test = locals()["final_X_test_" + frag_type[i]]
    frag_y_test = locals()["final_y_test_" + frag_type[i]]

    # Creating a local variable to hold predicted outputs for each frag type AND their trained models
    parts = "pred_parts_" + frag_type[i]
    models = "models_" + frag_type[i]
    locals()[parts] = []
    locals()[models] = []

    for cross in range(0,10):
        X_train = frag_X_train[cross]
        y_train = frag_y_train[cross]
        X_test = frag_X_test[cross]
        y_test = frag_y_test[cross]
        
        curr_R2s, curr_models = run_model_kf_cv(X_train, y_train, X_test,y_test, "parts")
        locals()[parts].append(curr_R2s)
        locals()[models].append(curr_models)


# len(within_bucket_R2s) # 10
# len(within_bucket_R2s[0]) # 16
# within_bucket_R2s[0][0] # each fold has R2s (vel,pos,acc) for each of the 16 buckets

In [23]:
# (NEW) Compute Within-Bucket combined XY_FVAF 
# For velocity only, although all kinematic variables should be used for model-fitting and predicted for.
from Neural_Decoding.metrics import compute_XY_FVAF

# 'pred_parts_AD' was thankfully also a list of arrays containing the R2s

for i in range(len(frag_type)):

    # Pulling local variables (computed R2s for parts)
    R2s_parts = locals()["pred_parts_" + frag_type[i]]

    # Creating a local variable to hold final XY_FVAFs
    XY_FVAFs = "XY_FVAFs_" + frag_type[i]
    locals()[XY_FVAFs] = []

    for j in range(0,10):
        curr_fold = R2s_parts[j]

        curr_fold_XY_FVAFs = []
        for k in range(0,16):
            #curr_bucket = Kalman_R2s_combined[i]
            vel_x_nom = curr_fold[k][0][0] # dim = (curr_bucket, nom, x_vel)
            vel_x_denom = curr_fold[k][1][0] # dim = (curr_bucket, denom, x_vel)
            vel_y_nom = curr_fold[k][0][1] # dim = (curr_bucket, nom, y_vel)
            vel_y_denom = curr_fold[k][1][1] # dim = (curr_bucket, denom, y_vel)

            XY_FVAF = compute_XY_FVAF(vel_x_nom,vel_x_denom,vel_y_nom,vel_y_denom)
            curr_fold_XY_FVAFs.append(XY_FVAF)

        locals()[XY_FVAFs].append(curr_fold_XY_FVAFs)

In [24]:
len(XY_FVAFs_Rand)
len(XY_FVAFs_Rand[0])

XY_FVAFs_Rand

10

16

[[-0.05895855148404672,
  0.010978191002686044,
  0.10898105720874696,
  0.15252195200948193,
  0.12347172654056748,
  -0.031815910410450954,
  0.13814005773006188,
  -0.12992251564687107,
  0.12175602870352797,
  0.14131231211390294,
  0.1197542515963329,
  0.1823304161448034,
  0.057876195903714955,
  0.03221324161886374,
  0.05581837340284246,
  0.08186843974526337],
 [0.15092966717990974,
  -0.09275244951760309,
  -0.0014020753443959588,
  0.1839716307426853,
  0.08280792239530432,
  0.10005861272219019,
  0.1299987521605943,
  0.09826350506714154,
  0.1690824906382108,
  0.14188143609057058,
  -0.08099711353613959,
  0.1375089915318154,
  0.08616030652559548,
  -0.11906328117216591,
  0.01013442089746519,
  0.16797685286419195],
 [0.06189112211975345,
  0.07928723156763495,
  0.007062119315380921,
  0.0632305167303786,
  0.0902631494705266,
  0.1748850042780259,
  0.08184488356907327,
  0.1504108106453631,
  0.1289821975512776,
  0.16676589340946724,
  0.0202084090911423,
  0.1363

## Cross-Bucket Tests

In [9]:
# (NEW) Opposite-polarity, Opposite-direction, Complete-opposite and Full cross-bucket tests
from Neural_Decoding.runModelsKF import opposite_polarity_test_cv, complete_opposite_bucket_test_cv, opposite_direction_test_cv, cross_buckets_test_cv

for i in range(len(frag_type)):

    # Pulling local variables 
    frag_X_test = locals()["final_X_test_" + frag_type[i]]
    frag_y_test = locals()["final_y_test_" + frag_type[i]]
    models = locals()["models_" + frag_type[i]]

    # Creating a local variable to hold FVAFs
    opposite_polarity = "opposite_polarity_FVAFs_" + frag_type[i]
    opposite_direction = "opposite_direction_FVAFs_" + frag_type[i]
    complete_opposite = "complete_opposite_FVAFs_" + frag_type[i]
    cross_buckets = "cross_buckets_FVAFs_" + frag_type[i]
    locals()[opposite_polarity] = []
    locals()[opposite_direction] = []
    locals()[complete_opposite] = []
    locals()[cross_buckets] = []


    for j in range(0,10):
        curr_models = models[j]
        curr_X_test = frag_X_test[j]
        curr_y_test = frag_y_test[j]

        # Creating a local variable to hold opposite-polarity and complete-opposite FVAFs
        locals()[opposite_polarity].append(opposite_polarity_test_cv(curr_models, curr_X_test, curr_y_test, "parts", frag_type[i]))
        locals()[opposite_direction].append(opposite_direction_test_cv(curr_models, curr_X_test, curr_y_test, "parts", frag_type[i]))
        locals()[complete_opposite].append(complete_opposite_bucket_test_cv(curr_models, curr_X_test, curr_y_test, "parts", frag_type[i]))
        locals()[cross_buckets].append(cross_buckets_test_cv(curr_models, curr_X_test, curr_y_test, "parts"))



In [13]:
# (NEW) Opposite-polarity, Opposite-direction
from Neural_Decoding.runModelsKF import opposite_polarity_test_cv, opposite_direction_test_cv

for i in range(len(frag_type)):

    # Pulling local variables 
    frag_X_test = locals()["final_X_test_" + frag_type[i]]
    frag_y_test = locals()["final_y_test_" + frag_type[i]]
    models = locals()["models_" + frag_type[i]]

    # Creating a local variable to hold FVAFs
    opposite_polarity = "opposite_polarity_FVAFs_" + frag_type[i]
    opposite_direction = "opposite_direction_FVAFs_" + frag_type[i]
    locals()[opposite_polarity] = []
    locals()[opposite_direction] = []

    for j in range(0,10):
        curr_models = models[j]
        curr_X_test = frag_X_test[j]
        curr_y_test = frag_y_test[j]

        # Creating a local variable to hold opposite-polarity and complete-opposite FVAFs
        locals()[opposite_polarity].append(opposite_polarity_test_cv(curr_models, curr_X_test, curr_y_test, "parts", frag_type[i]))
        locals()[opposite_direction].append(opposite_direction_test_cv(curr_models, curr_X_test, curr_y_test, "parts", frag_type[i]))

In [20]:
opposite_polarity_FVAFs_AD[0]
opposite_direction_FVAFs_AD[0]

[-0.596511754937795,
 -0.845377633796287,
 -0.5442052571835376,
 -0.6402684110153221,
 -0.8483529719343958,
 -0.5859790203411999,
 -0.8896922706650932,
 -0.579429367491493,
 -1.6662139961343092,
 -1.106784215383617,
 -0.522514916313269,
 -0.3016188123692447,
 -0.43763546376836904,
 -0.4207705093701606,
 -0.6720919195891486,
 -1.0339851259396013]

[-4.6676544329462235,
 -2.761034471579671,
 -3.476972390605133,
 -5.0486321668350165,
 -8.033987228362749,
 -5.708250846850241,
 -4.500824959062539,
 -4.03310552516325,
 -6.7133562521038685,
 -3.5940393315757104,
 -3.4620686490262083,
 -4.330984164207452,
 -5.151195240857863,
 -6.420394788951712,
 -4.713492347945885,
 -5.638184388273333]

### Part D: Partitioning and Running the Kalman Filter on Separate Training and Test Buckets

### Part E: Save all outputs (i.e. FVAFs)

In [23]:
# Saving results in a matlab file 
from scipy.io import savemat

result1 = [XY_FVAFs_AD, XY_FVAFs_Rand]
result2 = [cross_buckets_FVAFs_AD, cross_buckets_FVAFs_Rand]
result3 = [opposite_polarity_FVAFs_AD, opposite_polarity_FVAFs_Rand]
result4 = [opposite_direction_FVAFs_AD, opposite_direction_FVAFs_Rand]
result5 = [complete_opposite_FVAFs_AD, complete_opposite_FVAFs_Rand]

FrameStack1 = np.empty((2,), dtype=object)
FrameStack2 = np.empty((2,), dtype=object)
FrameStack3 = np.empty((2,), dtype=object)
FrameStack4 = np.empty((2,), dtype=object)
FrameStack5 = np.empty((2,), dtype=object)
for i in range(len(result1)):
    FrameStack1[i] = result1[i]
    FrameStack2[i] = result2[i]
    FrameStack3[i] = result3[i]
    FrameStack4[i] = result4[i]
    FrameStack5[i] = result5[i]
savemat("Within_Bucket.mat", {"XY_FVAF":FrameStack1})
savemat("Cross_Bucket.mat", {"Cross_Bucket":FrameStack2})
savemat("Opposite_Polarity.mat", {"Opposite_Polarity":FrameStack3})
savemat("Opposite_Direction.mat", {"Opposite_Direction":FrameStack4})
savemat("Complete_Opposite.mat", {"Complete_Opposite":FrameStack5})

### Notes

In [ ]:
for i in range(len(sortIn_AD)):
    X_kf = sortIn_AD[i]
    y_kf = decoder_output_AD[i]
    num_examples_kf=X_kf.shape[0] # nRows (b/c nCols = number of units)
    
    #Note that each range has a buffer of 1 bin at the beginning and end
    #This makes it so that the different sets don't include overlapping data
    training_set=np.arange(int(np.round(training_range[0]*num_examples_kf))+1,int(np.round(training_range[1]*num_examples_kf))-1)
    testing_set=np.arange(int(np.round(testing_range[0]*num_examples_kf))+1,int(np.round(testing_range[1]*num_examples_kf))-1)
    valid_set=np.arange(int(np.round(valid_range[0]*num_examples_kf))+1,int(np.round(valid_range[1]*num_examples_kf))-1)

    #Get training data
    X_kf_train=X_kf[training_set,:]
    y_kf_train=y_kf[training_set,:]

    #Get testing data
    X_kf_test=X_kf[testing_set,:]
    y_kf_test=y_kf[testing_set,:]

    #Get validation data
    X_kf_valid=X_kf[valid_set,:]
    y_kf_valid=y_kf[valid_set,:]

    #Z-score inputs 
    X_kf_train_mean=np.nanmean(X_kf_train,axis=0)
    X_kf_train_std=np.nanstd(X_kf_train,axis=0)
    X_kf_train=(X_kf_train-X_kf_train_mean)/X_kf_train_std
    X_kf_test=(X_kf_test-X_kf_train_mean)/X_kf_train_std
    X_kf_valid=(X_kf_valid-X_kf_train_mean)/X_kf_train_std

    #Zero-center outputs
    y_kf_train_mean=np.mean(y_kf_train,axis=0)
    y_kf_train=y_kf_train-y_kf_train_mean
    y_kf_test=y_kf_test-y_kf_train_mean
    y_kf_valid=y_kf_valid-y_kf_train_mean

    #Declare model
    model_kf=KalmanFilterDecoder(C=1) #There is one optional parameter (see ReadMe)

    #Fit model
    model_kf.fit(X_kf_train,y_kf_train)

    #Get predictions
    y_valid_predicted_kf=model_kf.predict(X_kf_valid,y_kf_valid)

    #Get metrics of fit (see read me for more details on the differences between metrics)
    #First I'll get the R^2
    R2_kf=get_R2(y_kf_valid,y_valid_predicted_kf)
    print('R2:',R2_kf[0:2]) #I'm just printing the R^2's of the 1st and 2nd entries that correspond to the positions
    #Next I'll get the rho^2 (the pearson correlation squared)